In [1]:
import os
import argparse
import time
from getpass import getuser
from distutils.dir_util import copy_tree
import pickle

import numpy as np
import random
import theano
import theano.tensor as T
from theano import config
import lasagne
from lasagne.regularization import regularize_network_params
from lasagne.objectives import categorical_crossentropy
from lasagne.objectives import categorical_accuracy

import PIL.Image as Image
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import gridspec

#from fcn_1D_general import buildFCN_1D
from metrics import accuracy, crossentropy
from data_loader.cortical_layers import ParcellationDataset #Cortical4LayersDataset, Cortical6LayersDataset
from classif_model import classif_model

%matplotlib inline

In [2]:
exp_name = 'simple_model_lrate=0.0005_fil=64_fsizes=[25]_depth=4_data=both_decay=0.001_pat=25'

#Model hyperparameters
n_filters = 64
filter_size = [25]
depth  = 4
data_augmentation={} #{'horizontal_flip': True, 'fill_mode':'constant'}
block = 'bn_relu_conv'

#Training loop hyperparameters
weight_decay=0.001
learning_rate_value = 0.0005 #learning rate is defined below as a theano variable.



#Hyperparameters for the dataset loader
batch_size=[1000,1000,1]
smooth_or_raw = 'both'
shuffle_at_each_epoch = True
minibatches_subset = 0


In [3]:
#SAVEPATH = '/Tmp/larocste/cortical_layers'
LOADPATH = '/data/lisatmp4/larocste/cortical_layers'
WEIGHTS_PATH = LOADPATH

#dataset = 'cortical_layers'
dataset='parcellation'
weight_path = os.path.join(WEIGHTS_PATH, dataset, exp_name, 'new_fcn1D_model_best.npz')

print 'weight path is ' , weight_path

weight path is  /data/lisatmp4/larocste/cortical_layers/parcellation/simple_model_lrate=0.0005_fil=64_fsizes=[25]_depth=4_data=both_decay=0.001_pat=25/new_fcn1D_model_best.npz


In [16]:
if smooth_or_raw =='both':
    nb_in_channels = 2
    use_threads = False
else:
    nb_in_channels = 1
    use_threads = True


train_iter = ParcellationDataset(
    which_set='train',
    smooth_or_raw = smooth_or_raw,
    batch_size=batch_size[0],
    data_augm_kwargs=data_augmentation,
    shuffle_at_each_epoch = shuffle_at_each_epoch,
    return_one_hot=False,
    return_01c=False,
    return_list=False,
    use_threads=use_threads)

valid_iter = ParcellationDataset(
    which_set='valid',
    smooth_or_raw = smooth_or_raw,
    batch_size=batch_size[1],
    shuffle_at_each_epoch = shuffle_at_each_epoch,
    return_one_hot=False,
    return_01c=False,
    return_list=False,
    use_threads=use_threads)

test_iter = None




n_batches_train = train_iter.nbatches
n_batches_val = valid_iter.nbatches
n_batches_test = test_iter.nbatches if test_iter is not None else 0
n_classes = train_iter.non_void_nclasses
void_labels = train_iter.void_labels

print 'nb batches train ' + str(n_batches_train) + ' valid ' + str(n_batches_val)



nb batches train 7 valid 2


In [5]:
#
# Define symbolic variables
#
input_var = T.tensor3('input_var') #n_example*nb_in_channels*ray_size
target_var = T.ivector('target_var') #n_example*ray_size

learn_step=  theano.shared(np.array(learning_rate_value, dtype=theano.config.floatX))

In [6]:
#
# Build model and assign trained weights
#
model = classif_model()

simple_net_output, net = model.build_model(input_var,
                    filter_size = filter_size,
                    n_filters = n_filters,
                    depth = depth,
                    block= block,
                    pool_factor = 2,
                    nb_in_channels = nb_in_channels,
                    n_classes = n_classes)

print 'Done building model'

with np.load(weight_path) as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    
nlayers = len(lasagne.layers.get_all_params(model.net))
lasagne.layers.set_all_param_values(model.net, param_values[:nlayers])

print 'Done assigning weights'

Done building model
Done assigning weights


In [7]:
print "Defining and compiling test functions"
test_prediction = lasagne.layers.get_output(model.net, deterministic=True)
test_loss = categorical_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
test_acc = T.mean(categorical_accuracy(test_prediction, target_var))


test_fn = theano.function([input_var, target_var], [test_loss, test_acc])
print "Done"

Defining and compiling test functions
Done


In [29]:
#Function computing the prediction with current parameters (for visualization)
pred = theano.function([input_var], lasagne.layers.get_output(model.net, deterministic=True))

In [65]:
def test_and_plot(exp_name, test_iter):
    
    
    n_batches_test = test_iter.nbatches
    cost_test_tot = 0
    acc_test_tot = 0
    
    
    n_true_labels = [0 for i in range(n_classes)]
    n_pred_labels = [0 for i in range(n_classes)]
    wrong_labels = [0 for i in range(n_classes)]
    wrong_pred_labels = [0 for i in range(n_classes)]
    
    for i in range(n_batches_test):
        
        # Get minibatch
        
        test_batch = test_iter.next()

        
        X_test_batch, L_test_batch, idx_test_batch = test_batch['data'], test_batch['labels'],test_batch['filenames'][0]   
        
        L_test_batch = np.reshape(L_test_batch, np.prod(L_test_batch.shape))
        
        nb = 50
        
        
        print 'True labels ' , L_test_batch[:nb]
        softmax_pred = pred(X_test_batch)
        #print 'softmax', softmax_pred
        
        #pred_labels = [softmax_pred[j][L_test_batch[j]] for j in range(nb)]
        #print 'Pred lables ' , pred_labels
        
        predicted_labels = np.array([np.argmax(softmax_pred[j]) for j in range(len(softmax_pred))])
        print 'Pred labels ', predicted_labels[:nb]
        print np.sum(predicted_labels[:nb]==L_test_batch[:nb])
        
        
        for label in L_test_batch:
            n_true_labels[label] += 1
        for label in predicted_labels :
            n_pred_labels[label] += 1
        for j in range(len(L_test_batch)):
            true_label = L_test_batch[j]
            pred_label = predicted_labels[j]
            if true_label!=pred_label:
                wrong_labels[true_label]+=1
                wrong_pred_labels[pred_label]+=1
        
        
        # Test step
        cost_test, acc_test = test_fn(X_test_batch, L_test_batch)
        
       
        acc_test_tot += acc_test
        cost_test_tot += cost_test
        
        

    err_test = cost_test_tot/n_batches_test
    acc_test = acc_test_tot/n_batches_test
    
        

    out_str = "FINAL MODEL: err test % f, acc test %f"
    out_str = out_str % (err_test,
                         acc_test)
    print exp_name
    print out_str
    
    print 'N_true labels : ', n_true_labels
    print 'N_pred labels : ', n_pred_labels
    print 'wrong labels : ', wrong_labels
    print 'wrong pred labels : ', wrong_pred_labels
    

In [62]:
test_and_plot(exp_name, train_iter)

True labels  [2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
Pred labels  [2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0


/part/01/Tmp/lisa/os_v5/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


True labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 1 1 1 1 1 1 1 1 1 1 1]
Pred labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 1 1 1 1 1 1 1 1 1 1 1]
0
True labels  [2 3 3 1 0 0 0 0 0 1 1 7 6 4 4 4 4 4 5 5 5 5 4 4 5 5 5 3 1 1 0 0 0 0 0 0 1
 1 7 6 6 4 4 4 4 5 5 4 4 4]
Pred labels  [2 3 3 1 0 0 0 0 0 1 1 7 6 4 4 4 4 4 5 5 5 5 4 4 5 5 5 3 1 1 0 0 0 0 0 0 1
 1 7 6 6 4 4 4 4 5 5 4 4 4]
0
True labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 1 1 1 1 1 1 1 1 1 1 1 1 1]
Pred labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
True labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 1 1 1]
Pred labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 1 1 1]
0
True labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 

In [66]:
test_and_plot(exp_name, valid_iter)

True labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 1 1 1 1 1 1 1 1 1 1 1 1]
Pred labels  [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 1 1 7 7 1 1 1 1 1 1 1 1 1]
47
True labels  [3 3 0 0 0 0 0 1 1 1 6 6 6 6 4 4 4 4 5 5 4 4 4 5 5 5 3 3 3 1 1 0 0 0 0 0 0
 0 0 1 6 6 6 4 4 4 5 4 4 4]
Pred labels  [3 3 6 0 0 0 0 1 1 1 6 6 6 6 4 4 4 4 5 5 4 4 4 5 5 5 3 2 3 1 5 0 0 0 0 0 0
 0 0 1 6 6 6 4 4 4 5 5 4 4]
46
simple_model_lrate=0.0005_fil=64_fsizes=[25]_depth=4_data=both_decay=0.001_pat=25
FINAL MODEL: err test  0.232856, acc test 0.911203
N_true labels :  [260, 128, 60, 70, 221, 157, 122, 56]
N_pred labels :  [266, 123, 63, 68, 221, 158, 124, 51]
wrong labels :  [5, 19, 1, 9, 14, 22, 8, 12]
wrong pred labels :  [11, 14, 4, 7, 14, 23, 10, 7]
